## Generating Annotations for Images

Uses the Google Cloud Vision API to generate annotations about what the content of the images are and visually simliar images.

In [24]:
import io
import os
import json
import queue

from google.cloud import vision
from google.protobuf.json_format import MessageToJson

from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

In [25]:
# Setting the API keys.
%env GOOGLE_APPLICATION_CREDENTIALS=C:\Users\Ethan\Desktop\repos\princeton-reverse-book-cover-search\api-keys\princeton-reverse-book-cover-5c6099bda6ff.json

env: GOOGLE_APPLICATION_CREDENTIALS=C:\Users\Ethan\Desktop\repos\princeton-reverse-book-cover-search\api-keys\princeton-reverse-book-cover-5c6099bda6ff.json


In [26]:
def annotate(path):
    """Returns web annotations given the path to an image.
    
    Uses Google Vision API to derive information about a
    given image from similar images on the web.
    
    Parameters
    ----------
    path : str
        Path to a file. Can be uri or filepath.
        
    Returns
    -------
    web_detection : google.cloud.vision_v1.types.web_detection.WebDetection
        API response that contains the returned information from the API.
    
    """
    client = vision.ImageAnnotatorClient()

    if path.startswith("http") or path.startswith("gs:"):
        image = vision.Image()
        image.source.image_uri = path

    else:
        with io.open(path, "rb") as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
    
    # FIXME: probably better to replace this with a batch request, but this is fine.
    web_detection = client.web_detection(image=image).web_detection

    return web_detection

In [ ]:
class BookCover:
    """Holds important information about the requests and responses to the GCP vision API.
    
    For each book cover avaliable, wraps the name of the book cover and response together as 
    a single unit. The response automatically serialized to allow for easy storage of 
    response in json format. 
    
    Attributes
    ----------
    image_file_name : str
        The file name of the image sent to GCP to annotate.
    raw_annotations : google.cloud.vision_v1.types.web_detection.WebDetection
        API response that contains the returned information from the API for the image.
    serialized_annotations : dict
        JSON form of response data.
    
    """
    def __init__(self, image_url):
        """Initalizes a BookCover object.
        
        Initalizes a BookCover object and automatically makes call to API for information
        about the input image.
        
        Parameters
        ----------
        image_url : str
            Either a statically hosted image url hosted on the internet or a 
            Google Cloud Storage Bucket
        
        """
        self.image_file_name = os.path.basename(image_url)
        self.raw_annotations = annotate(image_url)
        # Serializing the protobuf to store as json.
        self.serialized_annotations = json.loads(type(self.raw_annotations).to_json(self.raw_annotations))

    def __dict__(self):
        return {
            "image_name": self.image_file_name,
            "response": self.serialized_annotations,
        }

    def __str__(self):
        return json.dumps(self.__dict__())

    def __repr__(self):
        return json.dumps(self.__dict__())

#### Calling API on Each Image
Getting data for each image and storing it in a `BookCover` object, which just contains the filename and API reponse.

In [28]:
IMAGES_DIR = r"./data/all-book-set/covers"
images = os.listdir(IMAGES_DIR)
results = queue.Queue()
with (tqdm(total=len(images))) as pbar:
    with ThreadPoolExecutor(max_workers=16) as executor:
        for _ in executor.map(BookCover, [f'{IMAGES_DIR}/{image}' for image in images]):
            results.put(_)
            pbar.update(1)

  0%|          | 0/3625 [00:00<?, ?it/s]

#### Writing all Responses to File
Writing all the gathered data to an output file.

In [29]:
responses = []
while not results.empty():
    responses.append(results.get())

In [30]:
book_cover_data = {"data": [book_cover.__dict__() for book_cover in responses]}

In [31]:
OUTPUT_DIR = r"./data/all-book-set"
FILE_NAME = r"book_covers.json"
print("Writing to file...")
with open(f"{OUTPUT_DIR}/{FILE_NAME}", "w", encoding="utf-8") as outfile:
    # certain book titles have unicode characters in them, so we need to encode them as utf-8
    json.dump(book_cover_data, outfile, ensure_ascii=False)
print("Done!")

Writing to file...
Done!
